<a href="https://colab.research.google.com/github/HyunmookKim/zen-koan-nlp-analysis/blob/master/zen-koan-nlp-analysis/src%20/preprocessing/02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
선 공안 NLP 연구 - 전처리 Step 2.1: 공안
1) Blue Cliff Case 100 수정
2) Mumonkan 48 + Blue Cliff 100 파싱
"""

# =============================================================================
# CELL 1: 환경 설정
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

import os
import re
import json
import pandas as pd

BASE_PATH = '/content/drive/MyDrive/zen-koan-nlp-analysis'
RAW_PATH = f'{BASE_PATH}/data/extracted_texts_v10'
OUTPUT_PATH = f'{BASE_PATH}/data/processed'
CHUNKS_PATH = f'{BASE_PATH}/data/chunks'

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(CHUNKS_PATH, exist_ok=True)

print("환경 설정 완료!")

# =============================================================================
# CELL 2: Blue Cliff Case 100 수정
# =============================================================================

filepath = f"{RAW_PATH}/koans_blue_cliff.txt"

with open(filepath, 'r', encoding='utf-8') as f:
    content = f.read()

# 수정: === CASE === → === CASE 100 ===
if '=== CASE ===' in content:
    content = content.replace('=== CASE ===', '=== CASE 100 ===')
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)
    print("✅ Blue Cliff Case 100 수정 완료!")
else:
    print("ℹ️ 이미 수정되어 있음")

# 확인
markers = re.findall(r'=== CASE (\d+) ===', content)
print(f"Blue Cliff 총 케이스: {len(markers)}")

# =============================================================================
# CELL 3: spaCy 설정
# =============================================================================

!pip install -q spacy
!python -m spacy download en_core_web_sm

import spacy
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 3000000

print("spaCy 로드 완료!")

# =============================================================================
# CELL 4: 유틸리티 함수
# =============================================================================

def load_text_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    metadata = {}
    lines = content.split('\n')
    body_start = 0

    for i, line in enumerate(lines):
        if line.startswith('#'):
            if ':' in line:
                key, value = line[1:].split(':', 1)
                metadata[key.strip().lower()] = value.strip()
            body_start = i + 1
        else:
            break

    body = '\n'.join(lines[body_start:]).strip()
    return body, metadata

def tokenize_text(text):
    doc = nlp(text)
    tokens = [t for t in doc if not t.is_space]
    sentences = list(doc.sents)
    return len(tokens), len(sentences)

print("함수 정의 완료!")

# =============================================================================
# CELL 5: 공안 파싱
# =============================================================================

print("=" * 50)
print("공안 파싱 시작")
print("=" * 50)

koan_units = []
koan_full_text = ""
case_pattern = r'=== CASE (\d+) ==='

# --- Mumonkan ---
filepath = f"{RAW_PATH}/koans_mumonkan.txt"
text, metadata = load_text_file(filepath)

markers = list(re.finditer(case_pattern, text))
print(f"\nMumonkan 마커 수: {len(markers)}")

for i, match in enumerate(markers):
    case_num = int(match.group(1))
    start = match.end()
    end = markers[i+1].start() if i+1 < len(markers) else len(text)
    case_text = text[start:end].strip()

    if case_text:
        koan_full_text += case_text + "\n\n"
        token_count, sent_count = tokenize_text(case_text)
        koan_units.append({
            'category': 'koan',
            'text_name': 'mumonkan',
            'unit_id': f'mumonkan_case_{case_num}',
            'unit_num': case_num,
            'text': case_text,
            'token_count': token_count,
            'sentence_count': sent_count,
            'translator': 'Robert Aitken'
        })

mumonkan_count = len([u for u in koan_units if u['text_name'] == 'mumonkan'])
print(f"Mumonkan 파싱 완료: {mumonkan_count} cases")

# --- Blue Cliff Record ---
filepath = f"{RAW_PATH}/koans_blue_cliff.txt"
text, metadata = load_text_file(filepath)

markers = list(re.finditer(case_pattern, text))
print(f"\nBlue Cliff 마커 수: {len(markers)}")

for i, match in enumerate(markers):
    case_num = int(match.group(1))
    start = match.end()
    end = markers[i+1].start() if i+1 < len(markers) else len(text)
    case_text = text[start:end].strip()

    if case_text:
        koan_full_text += case_text + "\n\n"
        token_count, sent_count = tokenize_text(case_text)
        koan_units.append({
            'category': 'koan',
            'text_name': 'blue_cliff',
            'unit_id': f'blue_cliff_case_{case_num}',
            'unit_num': case_num,
            'text': case_text,
            'token_count': token_count,
            'sentence_count': sent_count,
            'translator': 'Thomas Cleary'
        })

blue_cliff_count = len([u for u in koan_units if u['text_name'] == 'blue_cliff'])
print(f"Blue Cliff 파싱 완료: {blue_cliff_count} cases")

# =============================================================================
# CELL 6: Level 1 저장
# =============================================================================

print("\n" + "=" * 50)
print("Level 1 저장")
print("=" * 50)

df_koan = pd.DataFrame(koan_units)

total_tokens = df_koan['token_count'].sum()
print(f"\n총 공안: {len(df_koan)} units")
print(f"총 토큰: {total_tokens:,}")
print(f"  - Mumonkan: {mumonkan_count} cases")
print(f"  - Blue Cliff: {blue_cliff_count} cases")

# JSON 저장
json_path = f"{OUTPUT_PATH}/koan_level1.json"
df_koan.to_json(json_path, orient='records', force_ascii=False, indent=2)
print(f"\n✅ JSON 저장: {json_path}")

# CSV 저장
csv_path = f"{OUTPUT_PATH}/koan_level1_meta.csv"
df_koan.drop(columns=['text']).to_csv(csv_path, index=False)
print(f"✅ CSV 저장: {csv_path}")

# =============================================================================
# CELL 7: Level 2 청크 생성
# =============================================================================

print("\n" + "=" * 50)
print("Level 2 청크 생성")
print("=" * 50)

CHUNK_SIZE = 1000
OVERLAP = 0.5

doc = nlp(koan_full_text)
tokens = [t.text for t in doc if not t.is_space]
print(f"전체 토큰 수: {len(tokens):,}")

chunks = []
stride = int(CHUNK_SIZE * (1 - OVERLAP))

for i in range(0, len(tokens) - CHUNK_SIZE + 1, stride):
    chunk_tokens = tokens[i:i + CHUNK_SIZE]
    chunks.append({
        'category': 'koan',
        'chunk_id': f'koan_chunk_{len(chunks)}',
        'chunk_num': len(chunks),
        'token_count': len(chunk_tokens),
        'text': ' '.join(chunk_tokens)
    })

print(f"생성된 청크: {len(chunks)}개")

df_chunks = pd.DataFrame(chunks)

json_path = f"{CHUNKS_PATH}/koan_level2.json"
df_chunks.to_json(json_path, orient='records', force_ascii=False, indent=2)
print(f"\n✅ JSON 저장: {json_path}")

csv_path = f"{CHUNKS_PATH}/koan_level2_meta.csv"
df_chunks.drop(columns=['text']).to_csv(csv_path, index=False)
print(f"✅ CSV 저장: {csv_path}")

# =============================================================================
# CELL 8: 최종 요약
# =============================================================================

print("\n" + "=" * 50)
print("📊 공안 전처리 완료")
print("=" * 50)
print(f"""
[Level 1 - 원본 단위]
  Mumonkan:    {mumonkan_count} cases
  Blue Cliff:  {blue_cliff_count} cases
  총계:        {len(koan_units)} units
  총 토큰:     {total_tokens:,}

[Level 2 - 청크]
  청크 수:     {len(chunks)}개
  청크 크기:   {CHUNK_SIZE} tokens
  오버랩:      {int(OVERLAP*100)}%
""")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
환경 설정 완료!
✅ Blue Cliff Case 100 수정 완료!
Blue Cliff 총 케이스: 100
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 64.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
spaCy 로드 완료!
함수 정의 완료!
공안 파싱 시작

Mumonkan 마커 수: 48
Mumonkan 파싱 완료: 48 cases

Blue Cliff 마커 수: 100
Blue Cliff 파싱 완료: 100 cases

Level 1 저장

총 공안: 148 units
총 토큰: 14,431
  - Mumonkan: 48 cases
  - Blue Cliff: 100 cases

✅ JSON 저장: /content/drive/MyDrive/zen-koan-nlp-analysis/data/processed/koan_level1.json
✅ CSV 저장: /content/drive/MyDrive/zen-koan-nlp-analysis/data/processed/koan_level1_meta.

In [2]:
"""
선 공안 NLP 연구 - 전처리 Step 2.2: 불교 경전 (Sutras)
"""

# =============================================================================
# CELL 1: 환경 설정
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

import os
import re
import json
import pandas as pd
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.max_length = 5000000

BASE_PATH = '/content/drive/MyDrive/zen-koan-nlp-analysis'
RAW_PATH = f'{BASE_PATH}/data/extracted_texts_v10'
OUTPUT_PATH = f'{BASE_PATH}/data/processed'
CHUNKS_PATH = f'{BASE_PATH}/data/chunks'

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(CHUNKS_PATH, exist_ok=True)

def load_text_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    metadata = {}
    lines = content.split('\n')
    body_start = 0
    for i, line in enumerate(lines):
        if line.startswith('#'):
            if ':' in line:
                key, value = line[1:].split(':', 1)
                metadata[key.strip().lower()] = value.strip()
            body_start = i + 1
        else:
            break
    body = '\n'.join(lines[body_start:]).strip()
    return body, metadata

def tokenize_text(text):
    doc = nlp(text)
    tokens = [t for t in doc if not t.is_space]
    sentences = list(doc.sents)
    return len(tokens), len(sentences)

print("환경 설정 완료!")

# =============================================================================
# CELL 2: 불교 경전 파싱
# =============================================================================

print("=" * 50)
print("불교 경전 파싱 시작")
print("=" * 50)

sutra_units = []
sutra_full_text = ""

SUTRA_FILES = [
    ('sutras_avatamsaka.txt', 'avatamsaka', 'Bhikshu Dharmamitra'),
    ('sutras_lankavatara.txt', 'lankavatara', 'D.T. Suzuki'),
    ('sutras_lotus.txt', 'lotus', 'BDK'),
    ('sutras_vimalakirti.txt', 'vimalakirti', 'Robert Thurman'),
    ('sutras_platform.txt', 'platform', 'Philip Yampolsky'),
    ('sutras_diamond.txt', 'diamond', 'A.F. Price'),
    ('sutras_amitabha.txt', 'amitabha', 'Various'),
    ('sutras_heart.txt', 'heart', 'Various'),
]

chapter_pattern = r'=== CHAPTER (\d+) ==='

for filename, name, default_translator in SUTRA_FILES:
    filepath = f"{RAW_PATH}/{filename}"
    if not os.path.exists(filepath):
        print(f"  ⚠ {filename} 없음")
        continue

    text, metadata = load_text_file(filepath)
    translator = metadata.get('translator', default_translator)
    sutra_full_text += text + "\n\n"

    markers = list(re.finditer(chapter_pattern, text))

    if markers:
        for i, match in enumerate(markers):
            ch_num = int(match.group(1))
            start = match.end()
            end = markers[i+1].start() if i+1 < len(markers) else len(text)
            ch_text = text[start:end].strip()

            if ch_text:
                token_count, sent_count = tokenize_text(ch_text)
                sutra_units.append({
                    'category': 'sutra',
                    'text_name': name,
                    'unit_id': f'{name}_ch_{ch_num}',
                    'unit_num': ch_num,
                    'text': ch_text,
                    'token_count': token_count,
                    'sentence_count': sent_count,
                    'translator': translator
                })
        print(f"  {name}: {len(markers)} chapters")
    else:
        token_count, sent_count = tokenize_text(text)
        sutra_units.append({
            'category': 'sutra',
            'text_name': name,
            'unit_id': f'{name}_full',
            'unit_num': 1,
            'text': text,
            'token_count': token_count,
            'sentence_count': sent_count,
            'translator': translator
        })
        print(f"  {name}: 1 unit (전체)")

print(f"\n총 경전 units: {len(sutra_units)}")

# =============================================================================
# CELL 3: Level 1 저장
# =============================================================================

print("\n" + "=" * 50)
print("Level 1 저장")
print("=" * 50)

df_sutra = pd.DataFrame(sutra_units)

print("\n경전별 통계:")
for name in df_sutra['text_name'].unique():
    subset = df_sutra[df_sutra['text_name'] == name]
    print(f"  {name}: {len(subset)} units, {subset['token_count'].sum():,} tokens")

total_tokens = df_sutra['token_count'].sum()
print(f"\n총계: {len(df_sutra)} units, {total_tokens:,} tokens")

json_path = f"{OUTPUT_PATH}/sutra_level1.json"
df_sutra.to_json(json_path, orient='records', force_ascii=False, indent=2)
print(f"\n✅ JSON 저장: {json_path}")

csv_path = f"{OUTPUT_PATH}/sutra_level1_meta.csv"
df_sutra.drop(columns=['text']).to_csv(csv_path, index=False)
print(f"✅ CSV 저장: {csv_path}")

# =============================================================================
# CELL 4: Level 2 청크 생성
# =============================================================================

print("\n" + "=" * 50)
print("Level 2 청크 생성")
print("=" * 50)

CHUNK_SIZE = 1000
OVERLAP = 0.5

print(f"전체 텍스트 길이: {len(sutra_full_text):,} 글자")

# 분할 토큰화 (메모리 문제 방지)
tokens = []
chunk_chars = 400000
for start in range(0, len(sutra_full_text), chunk_chars):
    chunk_text = sutra_full_text[start:start + chunk_chars]
    doc = nlp(chunk_text)
    tokens.extend([t.text for t in doc if not t.is_space])
    print(f"  토큰화 진행: {len(tokens):,} tokens")

print(f"전체 토큰 수: {len(tokens):,}")

chunks = []
stride = int(CHUNK_SIZE * (1 - OVERLAP))

for i in range(0, len(tokens) - CHUNK_SIZE + 1, stride):
    chunk_tokens = tokens[i:i + CHUNK_SIZE]
    chunks.append({
        'category': 'sutra',
        'chunk_id': f'sutra_chunk_{len(chunks)}',
        'chunk_num': len(chunks),
        'token_count': len(chunk_tokens),
        'text': ' '.join(chunk_tokens)
    })

print(f"생성된 청크: {len(chunks)}개")

df_chunks = pd.DataFrame(chunks)

json_path = f"{CHUNKS_PATH}/sutra_level2.json"
df_chunks.to_json(json_path, orient='records', force_ascii=False, indent=2)
print(f"\n✅ JSON 저장: {json_path}")

csv_path = f"{CHUNKS_PATH}/sutra_level2_meta.csv"
df_chunks.drop(columns=['text']).to_csv(csv_path, index=False)
print(f"✅ CSV 저장: {csv_path}")

# =============================================================================
# CELL 5: 최종 요약
# =============================================================================

print("\n" + "=" * 50)
print("📊 불교 경전 전처리 완료")
print("=" * 50)
print(f"""
[Level 1 - 원본 단위]
  총 경전: {len(df_sutra['text_name'].unique())}개
  총 units: {len(sutra_units)}
  총 토큰: {total_tokens:,}

[Level 2 - 청크]
  청크 수: {len(chunks)}개
""")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
환경 설정 완료!
불교 경전 파싱 시작
  avatamsaka: 39 chapters
  lankavatara: 4 chapters
  lotus: 28 chapters
  vimalakirti: 12 chapters
  platform: 10 chapters
  diamond: 32 chapters
  amitabha: 1 unit (전체)
  heart: 1 unit (전체)

총 경전 units: 127

Level 1 저장

경전별 통계:
  avatamsaka: 39 units, 944,225 tokens
  lankavatara: 4 units, 102,759 tokens
  lotus: 28 units, 94,291 tokens
  vimalakirti: 12 units, 45,274 tokens
  platform: 10 units, 35,411 tokens
  diamond: 32 units, 8,164 tokens
  amitabha: 1 units, 2,219 tokens
  heart: 1 units, 311 tokens

총계: 127 units, 1,232,654 tokens

✅ JSON 저장: /content/drive/MyDrive/zen-koan-nlp-analysis/data/processed/sutra_level1.json
✅ CSV 저장: /content/drive/MyDrive/zen-koan-nlp-analysis/data/processed/sutra_level1_meta.csv

Level 2 청크 생성
전체 텍스트 길이: 6,492,726 글자
  토큰화 진행: 71,418 tokens
  토큰화 진행: 148,245 tokens
  토큰화 진행: 223,753 tokens
  토큰화 진행

In [1]:
# ============================================
# STEP 2.3: 유교 경전 전처리 (중용 33장 업데이트)
# ============================================

import json
import re
import spacy
from pathlib import Path

# spaCy 로드
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2000000

BASE_PATH = Path("/content/drive/MyDrive/zen-koan-nlp-analysis")
RAW_PATH = BASE_PATH / "data" / "extracted_texts_v10"
PROCESSED_PATH = BASE_PATH / "data" / "processed"
CHUNKS_PATH = BASE_PATH / "data" / "chunks"

PROCESSED_PATH.mkdir(parents=True, exist_ok=True)
CHUNKS_PATH.mkdir(parents=True, exist_ok=True)

# 유교 텍스트 설정
CONFUCIAN_TEXTS = {
    'analects': {'file': 'confucian_analects.txt', 'translator': 'James Legge'},
    'mencius': {'file': 'confucian_mencius.txt', 'translator': 'James Legge'},
    'doctrine_mean': {'file': 'confucian_doctrine_mean.txt', 'translator': 'Robert Eno'},
    'filial_piety': {'file': 'confucian_filial_piety.txt', 'translator': 'Various'},
    'great_learning': {'file': 'confucian_great_learning.txt', 'translator': 'Robert Eno'}
}

def parse_confucian_text(text_name, content):
    """유교 텍스트별 파싱"""
    units = []

    if text_name == 'analects':
        # Book 마커: ^- [Name] \d+$
        pattern = r'^- ([A-Za-z\s]+) (\d+)\s*$'
        current_unit = None
        current_lines = []

        for line in content.split('\n'):
            match = re.match(pattern, line)
            if match:
                if current_unit and current_lines:
                    units.append({
                        'unit_id': f"analects_book_{current_unit}",
                        'unit_num': int(current_unit),
                        'text': '\n'.join(current_lines).strip()
                    })
                current_unit = match.group(2)
                current_lines = []
            elif current_unit:
                current_lines.append(line)

        if current_unit and current_lines:
            units.append({
                'unit_id': f"analects_book_{current_unit}",
                'unit_num': int(current_unit),
                'text': '\n'.join(current_lines).strip()
            })

    elif text_name == 'mencius':
        # Section 마커: ^- [Name]$ (숫자 없음)
        pattern = r'^- ([A-Za-z\s]+)\s*$'
        current_num = 0
        current_lines = []

        for line in content.split('\n'):
            match = re.match(pattern, line)
            if match:
                if current_num > 0 and current_lines:
                    units.append({
                        'unit_id': f"mencius_section_{current_num}",
                        'unit_num': current_num,
                        'text': '\n'.join(current_lines).strip()
                    })
                current_num += 1
                current_lines = []
            elif current_num > 0:
                current_lines.append(line)

        if current_num > 0 and current_lines:
            units.append({
                'unit_id': f"mencius_section_{current_num}",
                'unit_num': current_num,
                'text': '\n'.join(current_lines).strip()
            })

    elif text_name == 'doctrine_mean':
        # ★ 업데이트: === CHAPTER N === 마커 (33장)
        pattern = r'=== CHAPTER (\d+) ==='
        parts = re.split(pattern, content)

        for i in range(1, len(parts), 2):
            ch_num = int(parts[i])
            ch_text = parts[i + 1].strip() if i + 1 < len(parts) else ""
            if ch_text:
                units.append({
                    'unit_id': f"doctrine_mean_ch_{ch_num}",
                    'unit_num': ch_num,
                    'text': ch_text
                })

    elif text_name == 'filial_piety':
        # CHAPTER 마커: ^CHAPTER [WORD]:
        pattern = r'^CHAPTER ([A-Z]+):'
        word_to_num = {
            'ONE': 1, 'TWO': 2, 'THREE': 3, 'FOUR': 4, 'FIVE': 5,
            'SIX': 6, 'SEVEN': 7, 'EIGHT': 8, 'NINE': 9, 'TEN': 10,
            'ELEVEN': 11, 'TWELVE': 12, 'THIRTEEN': 13, 'FOURTEEN': 14,
            'FIFTEEN': 15, 'SIXTEEN': 16, 'SEVENTEEN': 17, 'EIGHTEEN': 18
        }
        current_unit = None
        current_lines = []

        for line in content.split('\n'):
            match = re.match(pattern, line)
            if match:
                if current_unit and current_lines:
                    units.append({
                        'unit_id': f"filial_piety_ch_{current_unit}",
                        'unit_num': current_unit,
                        'text': '\n'.join(current_lines).strip()
                    })
                word = match.group(1)
                current_unit = word_to_num.get(word, 0)
                current_lines = [line]
            elif current_unit:
                current_lines.append(line)

        if current_unit and current_lines:
            units.append({
                'unit_id': f"filial_piety_ch_{current_unit}",
                'unit_num': current_unit,
                'text': '\n'.join(current_lines).strip()
            })

    elif text_name == 'great_learning':
        # 단일 unit
        clean_text = '\n'.join([
            line for line in content.split('\n')
            if line.strip() and not line.startswith('#')
        ])
        units.append({
            'unit_id': 'great_learning_full',
            'unit_num': 1,
            'text': clean_text.strip()
        })

    return units

def process_text_with_spacy(text):
    """spaCy로 토큰화 및 문장 분리"""
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_space]
    sentences = [sent.text.strip() for sent in doc.sents]
    return tokens, sentences

def create_chunks(units, chunk_size=1000, overlap=500):
    """청크 생성"""
    all_tokens = []
    for unit in units:
        all_tokens.extend(unit['tokens'])

    chunks = []
    start = 0
    chunk_num = 1

    while start < len(all_tokens):
        end = min(start + chunk_size, len(all_tokens))
        chunk_tokens = all_tokens[start:end]

        chunks.append({
            'chunk_id': f"chunk_{chunk_num}",
            'chunk_num': chunk_num,
            'tokens': chunk_tokens,
            'token_count': len(chunk_tokens),
            'text': ' '.join(chunk_tokens)
        })

        chunk_num += 1
        start += overlap

        if end >= len(all_tokens):
            break

    return chunks

# 메인 처리
print("=" * 50)
print("유교 경전 전처리 시작 (중용 33장 업데이트)")
print("=" * 50)

all_units = []
total_tokens = 0

for text_name, info in CONFUCIAN_TEXTS.items():
    file_path = RAW_PATH / info['file']

    if not file_path.exists():
        print(f"⚠️ 파일 없음: {info['file']}")
        continue

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 파싱
    units = parse_confucian_text(text_name, content)
    print(f"\n{text_name}: {len(units)} units 파싱됨")

    # spaCy 처리
    for unit in units:
        tokens, sentences = process_text_with_spacy(unit['text'])
        unit['tokens'] = tokens
        unit['sentences'] = sentences
        unit['token_count'] = len(tokens)
        unit['sentence_count'] = len(sentences)
        unit['category'] = 'confucian'
        unit['text_name'] = text_name
        unit['translator'] = info['translator']
        total_tokens += len(tokens)

    all_units.extend(units)

    # 개별 통계
    unit_tokens = sum(u['token_count'] for u in units)
    print(f"  → {len(units)} units, {unit_tokens:,} tokens")

print(f"\n총 Level 1 units: {len(all_units)}")
print(f"총 tokens: {total_tokens:,}")

# Level 2 청크 생성
all_chunks = create_chunks(all_units, chunk_size=1000, overlap=500)
for chunk in all_chunks:
    chunk['category'] = 'confucian'
    chunk['text_name'] = 'confucian_combined'

print(f"Level 2 chunks: {len(all_chunks)}")

# Level 1 저장
level1_data = []
for unit in all_units:
    level1_data.append({
        'category': unit['category'],
        'text_name': unit['text_name'],
        'unit_id': unit['unit_id'],
        'unit_num': unit['unit_num'],
        'token_count': unit['token_count'],
        'sentence_count': unit['sentence_count'],
        'translator': unit['translator'],
        'text': unit['text']
    })

with open(PROCESSED_PATH / 'confucian_level1.json', 'w', encoding='utf-8') as f:
    json.dump(level1_data, f, ensure_ascii=False, indent=2)

# Level 1 메타 CSV
import csv
with open(PROCESSED_PATH / 'confucian_level1_meta.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['category', 'text_name', 'unit_id', 'unit_num', 'token_count', 'sentence_count', 'translator'])
    for unit in level1_data:
        writer.writerow([
            unit['category'], unit['text_name'], unit['unit_id'],
            unit['unit_num'], unit['token_count'], unit['sentence_count'], unit['translator']
        ])

# Level 2 저장
level2_data = []
for chunk in all_chunks:
    level2_data.append({
        'category': chunk['category'],
        'text_name': chunk['text_name'],
        'chunk_id': chunk['chunk_id'],
        'chunk_num': chunk['chunk_num'],
        'token_count': chunk['token_count'],
        'text': chunk['text']
    })

with open(CHUNKS_PATH / 'confucian_level2.json', 'w', encoding='utf-8') as f:
    json.dump(level2_data, f, ensure_ascii=False, indent=2)

# Level 2 메타 CSV
with open(CHUNKS_PATH / 'confucian_level2_meta.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['category', 'text_name', 'chunk_id', 'chunk_num', 'token_count'])
    for chunk in level2_data:
        writer.writerow([
            chunk['category'], chunk['text_name'], chunk['chunk_id'],
            chunk['chunk_num'], chunk['token_count']
        ])

print("\n" + "=" * 50)
print("✅ 유교 경전 전처리 완료!")
print("=" * 50)
print(f"Level 1: {len(all_units)} units → confucian_level1.json")
print(f"Level 2: {len(all_chunks)} chunks → confucian_level2.json")

# 텍스트별 요약
print("\n📊 텍스트별 요약:")
for text_name in CONFUCIAN_TEXTS.keys():
    text_units = [u for u in all_units if u['text_name'] == text_name]
    if text_units:
        tokens = sum(u['token_count'] for u in text_units)
        print(f"  {text_name}: {len(text_units)} units, {tokens:,} tokens")

유교 경전 전처리 시작 (중용 33장 업데이트)

analects: 20 units 파싱됨
  → 20 units, 34,254 tokens

mencius: 14 units 파싱됨
  → 14 units, 72,239 tokens

doctrine_mean: 33 units 파싱됨
  → 33 units, 6,262 tokens

filial_piety: 18 units 파싱됨
  → 18 units, 3,328 tokens

great_learning: 1 units 파싱됨
  → 1 units, 4,069 tokens

총 Level 1 units: 86
총 tokens: 120,152
Level 2 chunks: 240

✅ 유교 경전 전처리 완료!
Level 1: 86 units → confucian_level1.json
Level 2: 240 chunks → confucian_level2.json

📊 텍스트별 요약:
  analects: 20 units, 34,254 tokens
  mencius: 14 units, 72,239 tokens
  doctrine_mean: 33 units, 6,262 tokens
  filial_piety: 18 units, 3,328 tokens
  great_learning: 1 units, 4,069 tokens


In [1]:
"""
선 공안 NLP 연구 - 전처리 Step 2.4: 당송 산문 (Prose)
- Level 1: 57편 에세이 전체 (선별 없이)
- Level 2: 1000 tokens, 50% overlap 청크
"""

# =============================================================================
# CELL 1: 환경 설정
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

import os
import re
import json
import pandas as pd
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.max_length = 3000000

BASE_PATH = '/content/drive/MyDrive/zen-koan-nlp-analysis'
RAW_PATH = f'{BASE_PATH}/data/extracted_texts_v10'
OUTPUT_PATH = f'{BASE_PATH}/data/processed'
CHUNKS_PATH = f'{BASE_PATH}/data/chunks'

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(CHUNKS_PATH, exist_ok=True)

def tokenize_text(text):
    doc = nlp(text)
    tokens = [t for t in doc if not t.is_space]
    sentences = list(doc.sents)
    return len(tokens), len(sentences)

print("환경 설정 완료!")

# =============================================================================
# CELL 2: 산문 파싱
# =============================================================================

print("=" * 50)
print("당송 산문 파싱 시작")
print("=" * 50)

prose_units = []
prose_full_text = ""

filepath = f"{RAW_PATH}/prose_tang_song.txt"

with open(filepath, 'r', encoding='utf-8') as f:
    content = f.read()

print(f"파일 로드: {len(content):,} 문자")

# 에세이 마커 패턴: === ESSAY N: TITLE (AUTHOR, ERA) ===
essay_pattern = r'=== ESSAY (\d+): ([^(]+) \(([^,]+), (Tang|Song)\) ==='
markers = list(re.finditer(essay_pattern, content))
print(f"에세이 마커 수: {len(markers)}")

for i, match in enumerate(markers):
    essay_num = int(match.group(1))
    title = match.group(2).strip()
    author = match.group(3).strip()
    era = match.group(4).strip()

    start = match.end()
    end = markers[i+1].start() if i+1 < len(markers) else len(content)
    essay_text = content[start:end].strip()

    # 메타데이터 줄 제거 (# Author:, # Dates:, # Era:)
    lines = essay_text.split('\n')
    clean_lines = [line for line in lines if not line.startswith('#')]
    essay_text = '\n'.join(clean_lines).strip()

    if essay_text:
        prose_full_text += essay_text + "\n\n"
        token_count, sent_count = tokenize_text(essay_text)
        prose_units.append({
            'category': 'prose',
            'text_name': era.lower(),
            'unit_id': f'prose_essay_{essay_num}',
            'unit_num': essay_num,
            'title': title,
            'author': author,
            'era': era,
            'text': essay_text,
            'token_count': token_count,
            'sentence_count': sent_count,
            'translator': 'Herbert Giles'
        })

tang_count = len([u for u in prose_units if u['era'] == 'Tang'])
song_count = len([u for u in prose_units if u['era'] == 'Song'])
print(f"\n파싱 완료:")
print(f"  - 당(Tang): {tang_count}편")
print(f"  - 송(Song): {song_count}편")
print(f"  - 총계: {len(prose_units)}편")

# =============================================================================
# CELL 3: Level 1 저장
# =============================================================================

print("\n" + "=" * 50)
print("Level 1 저장")
print("=" * 50)

df_prose = pd.DataFrame(prose_units)

total_tokens = df_prose['token_count'].sum()
print(f"\n총 에세이: {len(df_prose)}")
print(f"총 토큰: {total_tokens:,}")

# 시대별 통계
print("\n시대별 통계:")
for era in ['Tang', 'Song']:
    subset = df_prose[df_prose['era'] == era]
    print(f"  {era}: {len(subset)}편, {subset['token_count'].sum():,} tokens")

# 저자별 통계
print("\n저자별 상위 10명:")
author_stats = df_prose.groupby('author').agg({
    'unit_num': 'count',
    'token_count': 'sum'
}).rename(columns={'unit_num': 'essays'}).sort_values('essays', ascending=False)
for author, row in author_stats.head(10).iterrows():
    print(f"  {author}: {int(row['essays'])}편, {int(row['token_count']):,} tokens")

# JSON 저장
json_path = f"{OUTPUT_PATH}/prose_level1.json"
df_prose.to_json(json_path, orient='records', force_ascii=False, indent=2)
print(f"\n✅ JSON 저장: {json_path}")

# CSV 저장 (text 제외)
csv_path = f"{OUTPUT_PATH}/prose_level1_meta.csv"
df_prose.drop(columns=['text']).to_csv(csv_path, index=False)
print(f"✅ CSV 저장: {csv_path}")

# =============================================================================
# CELL 4: Level 2 청크 생성
# =============================================================================

print("\n" + "=" * 50)
print("Level 2 청크 생성")
print("=" * 50)

CHUNK_SIZE = 1000
OVERLAP = 0.5

doc = nlp(prose_full_text)
tokens = [t.text for t in doc if not t.is_space]
print(f"전체 토큰 수: {len(tokens):,}")

chunks = []
stride = int(CHUNK_SIZE * (1 - OVERLAP))

for i in range(0, len(tokens) - CHUNK_SIZE + 1, stride):
    chunk_tokens = tokens[i:i + CHUNK_SIZE]
    chunks.append({
        'category': 'prose',
        'chunk_id': f'prose_chunk_{len(chunks)}',
        'chunk_num': len(chunks),
        'token_count': len(chunk_tokens),
        'text': ' '.join(chunk_tokens)
    })

print(f"생성된 청크: {len(chunks)}개")

df_chunks = pd.DataFrame(chunks)

json_path = f"{CHUNKS_PATH}/prose_level2.json"
df_chunks.to_json(json_path, orient='records', force_ascii=False, indent=2)
print(f"\n✅ JSON 저장: {json_path}")

csv_path = f"{CHUNKS_PATH}/prose_level2_meta.csv"
df_chunks.drop(columns=['text']).to_csv(csv_path, index=False)
print(f"✅ CSV 저장: {csv_path}")

# =============================================================================
# CELL 5: 최종 요약
# =============================================================================

print("\n" + "=" * 50)
print("📊 당송 산문 전처리 완료")
print("=" * 50)
print(f"""
[Level 1 - 원본 단위]
  당(Tang):    {tang_count}편
  송(Song):    {song_count}편
  총계:        {len(prose_units)} essays
  총 토큰:     {total_tokens:,}

[Level 2 - 청크]
  청크 수:     {len(chunks)}개
  청크 크기:   {CHUNK_SIZE} tokens
  오버랩:      {int(OVERLAP*100)}%

⚠️ 25편 선별은 분석 단계에서 진행
""")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
환경 설정 완료!
당송 산문 파싱 시작
파일 로드: 168,047 문자
에세이 마커 수: 57

파싱 완료:
  - 당(Tang): 19편
  - 송(Song): 38편
  - 총계: 57편

Level 1 저장

총 에세이: 57
총 토큰: 33,950

시대별 통계:
  Tang: 19편, 14,817 tokens
  Song: 38편, 19,133 tokens

저자별 상위 10명:
  SU TUNG-P'O: 11편, 7,364 tokens
  HAN WEN-KUNG: 8편, 7,695 tokens
  OU-YANG HSIU: 7편, 4,435 tokens
  LIU TSUNG-YUAN: 6편, 4,133 tokens
  SHEN KUA: 3편, 784 tokens
  CHU HSI: 2편, 1,216 tokens
  SUNG TZ'U: 2편, 339 tokens
  LO KUAN-CHUNG: 2편, 1,743 tokens
  WANG AN-SHIH: 2편, 592 tokens
  LI CHIH: 1편, 421 tokens

✅ JSON 저장: /content/drive/MyDrive/zen-koan-nlp-analysis/data/processed/prose_level1.json
✅ CSV 저장: /content/drive/MyDrive/zen-koan-nlp-analysis/data/processed/prose_level1_meta.csv

Level 2 청크 생성
전체 토큰 수: 33,950
생성된 청크: 66개

✅ JSON 저장: /content/drive/MyDrive/zen-koan-nlp-analysis/data/chunks/prose_level2.json
✅ CSV 저장: /content/drive/MyDrive